In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from EDAModule.Initialize import *
from EDAModule.State import State

In [ ]:
initialize()